# Chemical equilibrium with constraints

This tutorial demonstrates how Reaktoro can be used for a wide range of equilibrium problems and explains how to

* *specify different equilibrium constraints* when calculating chemical equilibrium and
* *configure equilibrium solver* to be open to one or more substances to satisfy these constraints.

Let us re-call the simplest chemical equilibrium discussed earlier, where:
* T and P are given and
* the system is closed (no mass transfer in or out of the system).

In [ ]:
from reaktoro import *

db = PhreeqcDatabase("llnl.dat")

gases = GaseousPhase("CH4(g) O2(g) CO2(g) CO(g) H2O(g) H2(g)")

system = ChemicalSystem(db, gases)

solver = EquilibriumSolver(system) # notat that EquilibriumSolver is intialized with chemical system

To impose the **temperature** and **volume** of the system (technically the class of *Helmholtz energy minimization problems*), we need to construct *a specialized [EquilibriumSolver](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSolver.html) object* that handles alternative constraint specifications.

> **Note**: Instead of the Helmholtz energy minimization problem, Reaktoro implements a single **parametric Gibbs energy minimization solver that accepts general equilibrium constraints**, which can be configured to efficiently solve all other classes of equilibrium problems.


## Specifying open conditions

Assume, we need to find out how many mols of CH<sub>4</sub> must react with 0.5 mol of O<sub>2</sub> in the same chamber (with **volume** 10 cm³ and **temperature** 1000 °C) to obtain **pressure** 1 GPa (maximum pressure supported by the chamber material). In such equilibrium calculation, the system must be assumed to be open to the mass transfer of CH<sub>4</sub>, which creates a new *degree of freedom*: the amount of added/removed substance.

We start with the chemical state containing only 0.5 mols of O<sub>2</sub>:

In [ ]:
state = ChemicalState(system)
state.set("O2(g)", 0.5, "mol")

Next, we create an [EquilibriumSpecs](https://reaktoro.org/api/classReaktoro_1_1EquilibriumSpecs.html) object specifying **temperature**, **volume**, and **pressure** as constrained properties. We also specify that the system is open to CH<sub>4</sub>.

In [ ]:
specs = EquilibriumSpecs(system)
specs.temperature()
specs.volume()
specs.pressure()
specs.openTo("CH4")

Then, we create the [EquilibriumConditions](https://reaktoro.org/api/classReaktoro_1_1EquilibriumConditions.html) object, providing the values for temperature, volume, and pressure, and perform the equilibrium calculation:

In [ ]:
conditions = EquilibriumConditions(specs)
conditions.temperature(25.0, "celsius")
conditions.volume(10.0, "cm3")
conditions.pressure(1.0, "GPa")

solver = EquilibriumSolver(specs)
solver.solve(state, conditions)

print(state.props())

The above table shows that the obtained equilibrium state meats desired values for temperature, pressure, and volume (in SI units). We can output the amount of CH<sub>4</sub> entering the system so that all prescribed conditions could be attained:

In [ ]:
print("Amount of CH4 titrated in:", float(state.equilibrium().explicitTitrantAmounts()), "mol")